## MACD+Parameter+Optimization+(c)

In [1]:
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd 
import pandas_datareader.data as pdr
import numpy as np

In [2]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

def calculate_macd(prices, fast, slow, smoothing=2):
    fast_ema=pd.DataFrame(calculate_ema(prices,fast),index=np.arange(fast,len(prices)+1),columns=['fast_ema'])
    slow_ema=pd.DataFrame(calculate_ema(prices,slow),index=np.arange(slow,len(prices)+1),columns=['slow_ema'])
    df_macd = pd.concat([fast_ema,slow_ema], axis=1, join="outer")[:-1]
    df_macd['macd']=df_macd['fast_ema']-df_macd['slow_ema']
    df_macd=df_macd[['macd']]
    return df_macd

In [3]:
def macd_backtesting(ticker, fast, slow, previous_days):
    
    global df
    df_signal = ohlc_intraday[ticker].copy()
    df_signal = df_signal[['Date','Close']]

    macd=calculate_macd(df_signal["Close"],fast,slow)
    df_signal = pd.concat([df_signal,macd],axis=1,join='outer')[:-1]
    condition =  df_signal['macd'] > 0
    
    df_signal['Hold']=np.where(condition, 'Hold', '')
    df_signal['Hold_t-1']=df_signal['Hold'].shift(1)
    df_signal['Hold_t-2']=df_signal['Hold'].shift(2)

    buy_condition = (df_signal['Hold_t-2'] =='') & (df_signal['Hold_t-1'] =='Hold') 
    df_signal['Buy']=np.where(buy_condition, 'Buy', '')

    sell_condition = (df_signal['Hold_t-2'] =='Hold') & (df_signal['Hold_t-1'] =='') 
    df_signal['Sell']=np.where(sell_condition, 'Sell', '')
    df_signal = df_signal[['Date','Buy','Sell']]
    condition = (df_signal['Buy']=='Buy') | (df_signal['Sell']=='Sell')
    df_signal = df_signal[condition]
    df_signal = df_signal.reset_index(drop = True)
    
    columns_Trading = ['Date_Buy','Price_Buy','Date_Sell','Price_Sell','PL(%)','Holding_Days','Accumulated profit']
    df_Trading = pd.DataFrame(columns = columns_Trading)
    
    Accu_profit = 0
    for i in range(0,len(df_signal),2):
        if(i<len(df_signal)-1):
            if (df_signal['Buy'][i]=='Buy') & (df_signal['Sell'][i+1]=='Sell'):

                Date_Buy = df_signal['Date'][i]
                Date_Sell = df_signal['Date'][i+1]
                Price_Buy = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Buy][['Open']].values[0][0]
                Price_Sell = ohlc_intraday[ticker][ohlc_intraday[ticker]['Date']==Date_Sell][['Open']].values[0][0]
                fee = 0.002 * (Price_Buy + Price_Sell)
                Holding_Days = abs(Date_Sell-Date_Buy).days
                PL = (Price_Sell-Price_Buy-fee)*100/Price_Buy
                Accu_profit += PL

                Add_Trading = pd.DataFrame([[Date_Buy,Price_Buy,Date_Sell,Price_Sell,PL,Holding_Days,Accu_profit]],
                                           columns = columns_Trading)
                df_Trading = df_Trading.append(Add_Trading)

    df_Trading = df_Trading.reset_index(drop = True)
    
    total_profit = df_Trading['Accumulated profit'][len(df_Trading)-1]
    cagr = ((((total_profit+100)/100)**(1/(len(ohlc_intraday[ticker])/250)))-1)*100
    win_trade = df_Trading[df_Trading["PL(%)"]>0].count()["PL(%)"]
    loss_trade =  df_Trading[df_Trading["PL(%)"]<=0].count()["PL(%)"]
    win_rate = win_trade*100/(win_trade+loss_trade)
    avg_gain = df_Trading[df_Trading["PL(%)"]>0]["PL(%)"].mean()
    avg_loss = df_Trading[df_Trading["PL(%)"]<=0]["PL(%)"].mean()
    expected_return = (win_rate*avg_gain/100)+((1-win_rate)*avg_loss/100)
    add_df = pd.DataFrame([[fast,slow,total_profit,cagr,win_trade,loss_trade,win_rate,avg_gain,avg_loss,
                            expected_return]],columns=column)
    df = df.append(add_df)
    

In [4]:
ticker ='AMZN'
previous_days = 4000

column = ['Fast','Slow','Total Profit','CAGR','Win Trade','Loss Trade','Win Rate',
          'Average Gain','Average loss','Expected Return']
df = pd.DataFrame(columns =column)

start = dt.datetime.today()-dt.timedelta(previous_days)
end = dt.datetime.today()
ohlc_intraday = {}
ohlc_intraday[ticker] = yf.download(ticker,start,end)
ohlc_intraday[ticker] = ohlc_intraday[ticker].reset_index(drop = False)

for s in range(6,201,3):
    for f in range(3,s,3):
        macd_backtesting(ticker, fast=f, slow=s, previous_days =previous_days)
df = df.sort_values(by = ['Total Profit'],ascending = False).reset_index(drop =True)

[*********************100%***********************]  1 of 1 completed


In [5]:
df.round(2)

,Fast,Slow,Total Profit,CAGR,Win Trade,Loss Trade,Win Rate,Average Gain,Average loss,Expected Return
0,105,159,416.83,16.05,4,1,80.00,104.87,-2.66,86.00
1,45,159,416.50,16.05,5,2,71.43,85.34,-5.10,64.55
2,108,156,413.77,15.99,3,2,60.00,139.46,-2.31,85.04
3,108,159,413.24,15.98,3,1,75.00,138.26,-1.54,104.83
4,66,120,412.52,15.97,3,2,60.00,142.13,-6.93,89.37
...,...,...,...,...,...,...,...,...,...,...
2140,105,177,-6.90,-0.65,1,3,25.00,33.75,-13.55,11.69
2141,102,183,-7.70,-0.72,1,3,25.00,33.10,-13.60,11.54
2142,90,195,-8.02,-0.76,1,3,25.00,34.65,-14.22,12.08
2143,87,198,-8.40,-0.79,1,3,25.00,34.25,-14.22,11.97


In [6]:
df.sort_values(by = ['Expected Return'],ascending = False).reset_index(drop =True)

,Fast,Slow,Total Profit,CAGR,Win Trade,Loss Trade,Win Rate,Average Gain,Average loss,Expected Return
0,84,177,369.358472,15.045280,3,1,75.0,131.315898,-24.589221,116.682946
1,93,171,392.322540,15.544493,3,1,75.0,136.977656,-18.610428,116.504959
2,129,135,388.452942,15.461876,3,1,75.0,135.244484,-17.280509,114.220940
3,123,141,388.799199,15.469293,3,1,75.0,135.244484,-16.934252,113.964709
4,114,159,402.726719,15.763730,3,1,75.0,138.614398,-13.116474,113.666990
...,...,...,...,...,...,...,...,...,...,...
2140,174,177,20.051618,1.670356,1,0,100.0,20.051618,NaN,NaN
2141,156,195,19.559997,1.632546,1,0,100.0,19.559997,NaN,NaN
2142,159,198,19.143194,1.600379,1,0,100.0,19.143194,NaN,NaN
2143,156,198,19.143194,1.600379,1,0,100.0,19.143194,NaN,NaN
